## label extraction


In [ ]:
#coding:utf-8
#used for label extraction
#2018.06.06
#+—————zh_char:(path_zh)
#+     |—————name=川，湘，...(path_zh_folder)
#+           |—————file_=**.jpg....(dir)
#+—————alphabet_number
#############################################

import os, sys, stat
import pandas as pd
import numpy as np
path_zh='./Segment_Dataset/zh_char/'
path_ch='./Segment_Dataset/ch_/'
path_alphabet='./Segment_Dataset/alphabet_number/'

#reset image file name
zh_=os.listdir(path_zh)#all file/folder in path
for name in np.array(zh_):
    path_zh_folder=path_zh+name
    #os.chmod(path_zh_folder,stat.S_IRWXU)
    filelist=os.listdir(path_zh_folder)#image files name in zh_folder,image name+.jpg
    i=0
    for file_ in filelist:
        olddir=os.path.join(path_zh_folder,file_)
        filename=os.path.splitext(file_)[0]
        filetype=os.path.splitext(file_)[1]
        newdir=os.path.join(path_ch,name+str(i)+filetype)
        os.rename(olddir,newdir)
        i=i+1


## Label2Txt

In [ ]:
#coding:utf-8
#write label into txt
#2018.06.06 
#############################################
import os 

path_ch='./test_data/ch_/'
path_number='./test_data/number_'

with open ("./test_data/test_label_ch.txt",'a') as f:
    filelist=os.listdir(path_ch)
    for file_ in filelist:
        filename=os.path.splitext(file_)[0]
        label=filename[0:3]#eg one chinese char
        filetype=os.path.splitext(file_)[1]
        f.write(filename+':'+label+'\n')



## @Multiclass SVM

In [ ]:
#coding:utf-8
#Creat an multiclass svm and train
#2018.06.07
#############################################
import cv2 as cv 
import pandas as pd 
import numpy as np 
import os
import logging
import time, timeit

def ini():  
    TM=time.strftime("%Y-%m-%d %H-%M-%S",time.localtime())
    LOG_FORMAT = "%(levelname)s -[:%(lineno)d]- %(message)s"
    #open(log_nm,'w').close()
    logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
    logging.info('**************************Mason(%s)*******************'%(TM,))

    
    
    


#create dataset object and label object func
def load_data(path_X,path_Y):#path_X is folder and path_Y is a txt file
    X=[]
    Y=[]
    filelist=os.listdir(path_X)
    df=pd.read_table(path_Y,sep=":",header=None)#,encoding='utf-8'
    df.columns=['name','label']
    for file_ in filelist:
        #read image
        path_image=os.path.join(path_X,file_)
        image=cv.imread(path_image,0)#must be gray model
        #unisize
        image=cv.resize(image,(20,20))
        
        feature=image
        #image-->list
        try:
            N_pixels=(np.array(feature).shape[0])*(np.array(feature).shape[1])
        except IndexError:
            logging.error("image read error! feature shape is:"%(np.array(feature).shape))
            raise       
        
        image_vector=np.reshape(np.array(feature),N_pixels)
        #logging.info('vector length of image %s: %d'%(file_,np.array(image_vector.shape)))
        X.append(image_vector.tolist())
        #index label
        filename=os.path.splitext(file_)[0]
        label=df.loc[df['name']==filename,'label']
        Y.append(label.tolist())
    return X,Y #X is dataset and Y is label related




#train ch_ /alphabet_number
if __name__=='__main__':
    ini()
    path_X_train='./Segment_Dataset/train_data/ch_'
    path_Y_train='./Segment_Dataset/train_data/train_label_ch2.txt'
    #get dataset and label object
    logging.info('feature extract and load data...')
    load_begin= timeit.default_timer()  
    
    X_train,Y_train=load_data(path_X_train,path_Y_train)
    
    #logging.info(np.array(Y_train).shape) 
    load_end = timeit.default_timer()  
    logging.info('X_train shape:(%d,%d)'%(np.array(X_train).shape[0],np.array(X_train).shape[1]))
    logging.info('Y_train(labels) shape:(%d,%d)'%(np.array(Y_train).shape[0],np.array(Y_train).shape[1]))
    logging.info('time consuption of load data:%0.1f second'%(load_end-load_begin))
    
    #create svm object
    logging.info('create svm...')
    svm=cv.ml.SVM_create()
    svm.setKernel(cv.ml.SVM_LINEAR)#cv.ml.SVM_LINEAR,
    svm.setType(cv.ml.SVM_C_SVC)#cv.ml.SVM_EPS_SVR,  cv.ml.SVM_C_SVC
    svm.setGamma(2.0)
    svm.setC(5.0)
    svm.setP(1.0)#must postive for SVM_EPS_SVR type
    #svm.setNu(0.5)# only for SVM_NU_SVC type and between 0-1
    #training &save svm
    logging.info('training...')
    train_begin= timeit.default_timer() 
    svm.train(np.array(X_train,dtype=np.float32),cv.ml.ROW_SAMPLE,np.array(Y_train))
    load_end= timeit.default_timer() 
    logging.info('time consuption of training:%0.4f second'%(load_end-load_begin))
    
    logging.info('save the trained svm...')

    svm.save('./svm_model.m')#‘path’

    #predicate 
    logging.info('Done!')
    #Y_test_predict=svm.predict(np.array(X_test,dtype=np.float32))




## @Single Image Predict

In [ ]:
#coding:utf-8
#load svm model and just for predict single image
#############################################

import cv2 as cv 
import numpy as np
import pandas as pd





df=pd.read_table('./Segment_Dataset/mapping.txt',sep=":",header=None)#,encoding='utf-8'
df.columns=['ch','nb']
X_=[]
zh=[]
extract=0
if extract==1:
    svm=cv.ml.SVM_load('./new_svm_model.m')
else:
    svm=cv.ml.SVM_load('./svm_model.m')
image=cv.imread('./Segment_Dataset/粤_.png',cv.IMREAD_GRAYSCALE)
image=cv.resize(image,(20,20))

feature=image
N=(np.array(feature).shape[1])*(np.array(feature).shape[0])

img=np.reshape(np.array(feature),N)

X_.append(img.tolist())# X_train is a list; can not use extend
Y_predict=svm.predict(np.array(X_,dtype=np.float32),cv.ml.ROW_SAMPLE)
result=int(Y_predict[1][0][0])
print result
ch=df.loc[df['nb']==result,'ch']
zh.append(ch.tolist())# an 2D array
print np.array(zh)[0][0]




## PCA-Feature Extract

In [ ]:
#coding:utf-8
#adding PCA calculation
#2018/06/11

import numpy as np
import cv2 as cv 

image=cv.imread('./Segment_Dataset/粤_5.bmp',cv.IMREAD_GRAYSCALE)
image=cv.resize(image,(20,20))
mat_=np.array(image)#40*32
print np.array(mat_).shape
#Mean & Normalization 
Mean=[np.mean(mat_[:,i]) for i in range(0,mat_.shape[1])]
Z=[np.array(mat_[:,i]-Mean[i]) for i in range(0,mat_.shape[1])]
'''#equal to s
Z=[]
for i in range(0,mat_.shape[1]):# shape[0] is hight of image in numpy (h,w) 
    M=np.mean(mat_[:,i])# mean value of each column
    Z.append(mat_[:,i]-M)# normalized '''
    
data=np.transpose(Z)
#data=Z
print 'shape of data is {}'.format(np.array(data).shape)
# Covariance
cov=np.cov(data)#32*32
#print cov
# calculating eigenvalue&vector
eig_val, eig_vector = np.linalg.eig(cov)

print 'shape of eig_vector: {}'.format(np.array(eig_vector).shape)
print 'shape of eig_val: {}'.format(np.array(eig_val).shape)
eig_pairs = [(np.abs(eig_val[i]), eig_vector[:,i]) for i in range(len(eig_val))]

#select the top k eig_
index_=np.argsort(eig_val)#totally 32 eig_pairs

# select higher 11 eig
eig_val_n=[np.abs(eig_val[i]) for i in index_[0:11:1]]
print eig_val_n
eig_vector_n=[(eig_vector[:,i]) for i in index_[0:10:1]]
print 'shape of selected eig_val: {}'.format(np.array(eig_val_n).shape )
print 'shape of selected eig_vector: {}'.format(np.array(eig_vector_n).shape )
#reconstructs new data
new_data=np.dot(eig_vector_n,data) # data*eigenvector 所谓降维，降的是h 的维度，也就是row 的数量
print 'shape of new_data: {}'.format(np.array(new_data).shape )
#print  new_data



    




